# Big Data and Data Analytics for Managers 

In [ ]:
Corona_NLP_BDDA.ipynb

## Merlin John - 015027

AIM- Perform Text Classification on Coronavirus tweets using Py spark


Columns:

1) Location
2) Tweet At
3) Original Tweet
4) Label

In [ ]:
#!pip install pyspark

In [ ]:
#import os       #importing os to set environment variable
#def install_java():
#  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
#  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
#  !java -version       #check java version
#install_java()

In [1]:
#from pyspark.sql import SparkSession
from pyspark.sql.functions import length,  regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer,HashingTF
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline

In [ ]:
#spark = SparkSession.builder \
#        .config("spark.driver.memory","6G")\
#        .config("spark.driver.maxResultSize", "4G") \
#        .getOrCreate()

In [ ]:
#from google.colab import drive
#drive.mount('/gdrive')

In [ ]:
#path = "/gdrive/MyDrive/Colab_data_files/corona_nlp/Corona_NLP_test.csv"
#data=spark.read.csv(path, header = True, inferSchema=True)

In [ ]:
! hdfs dfs -mkdir -p /user/ashok/data_files/corona
! hdfs dfs -put /home/ashok/Documents/corona_nlp_test.csv  /user/ashok/data_files/corona
! hdfs dfs -ls /user/ashok/data_files/corona

In [2]:
! hdfs dfs -mkdir -p /user/ashok/data_files/corona
! hdfs dfs -put /home/ashok/Downloads/corona_nlp_test.csv  /user/ashok/data_files/corona
! hdfs dfs -ls /user/ashok/data_files/corona

put: `/user/ashok/data_files/corona/corona_nlp_test.csv': File exists
Found 1 items
-rw-r--r--   1 ashok supergroup    1002494 2021-11-12 14:41 /user/ashok/data_files/corona/corona_nlp_test.csv


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [4]:
path = "/user/ashok/data_files/corona/corona_nlp_test.csv"
data=spark.read.csv(path, header = True, inferSchema=True)

In [5]:
data.show()
data.printSchema()

+--------------------+--------------------+--------------------+----------+--------------------+------------------+
|            UserName|          ScreenName|            Location|   TweetAt|       OriginalTweet|         Sentiment|
+--------------------+--------------------+--------------------+----------+--------------------+------------------+
|                   1|               44953|                 NYC|02-03-2020|TRENDING: New Yor...|Extremely Negative|
|                   2|               44954|         Seattle, WA|02-03-2020|When I couldn't f...|          Positive|
|                   3|               44955|                null|02-03-2020|Find out how you ...|Extremely Positive|
|                   4|               44956|         Chicagoland|02-03-2020|#Panic buying hit...|              null|
|https://t.co/IASi...|                null|                null|      null|                null|              null|
|#QAnon #QAnon2018...|                null|                null|      nu

In [6]:
data.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|            UserName|          ScreenName|            Location|             TweetAt|       OriginalTweet|           Sentiment|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  count|                6792|                5340|                3180|                3866|                3826|                2533|
|   mean|   1896.592797055731|  46766.095926412614|  16318.284598214286|                null|                null|                null|
| stddev|  1098.1117587571187|   2271.481448081071|      33413.20664644|                null|                null|                null|
|    min|                 ...|  we must DEMAND ...|                    | #sainsburys etc....|  We ve got sever...| ""buy tinned foo...|
|    max|          ··········|       @CNN Busine

# Data Prepration

In [7]:
result_df = data.groupBy("Location").count().sort("Location", ascending=False)
result_df.show(10)

+--------------------+-----+
|            Location|count|
+--------------------+-----+
|ÜT: 44.881667,-93...|    1|
|ÜT: 43.661815,-79...|    1|
|ÜT: 43.64624,-79....|    1|
|ÜT: 40.725815,-74...|    1|
|ÜT: 40.5896566,-7...|    1|
|ÜT: 3.099457,101....|    1|
|ÜT: 28.508488,-81...|    1|
| ~ JHB, South Africa|    1|
|    www.303Gonzo.com|    1|
|       world citizen|    1|
+--------------------+-----+
only showing top 10 rows



In [8]:
data=data.withColumn('length', length(data['OriginalTweet']))
data.show(10)

+--------------------+----------+-------------------+----------+--------------------+------------------+------+
|            UserName|ScreenName|           Location|   TweetAt|       OriginalTweet|         Sentiment|length|
+--------------------+----------+-------------------+----------+--------------------+------------------+------+
|                   1|     44953|                NYC|02-03-2020|TRENDING: New Yor...|Extremely Negative|   228|
|                   2|     44954|        Seattle, WA|02-03-2020|When I couldn't f...|          Positive|   193|
|                   3|     44955|               null|02-03-2020|Find out how you ...|Extremely Positive|    73|
|                   4|     44956|        Chicagoland|02-03-2020|#Panic buying hit...|              null|   209|
|https://t.co/IASi...|      null|               null|      null|                null|              null|  null|
|#QAnon #QAnon2018...|      null|               null|      null|                null|              null|

In [9]:
data=data.drop("UserName","ScreenName","Location","TweetAt")
data.filter(data["Sentiment"]=="Negative").show(10)

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|#CoronaVirus #COV...| Negative|   284|
|When your usual g...| Negative|   201|
|While you are sto...| Negative|   250|
|Two important pho...| Negative|   271|
|Fellow Uni instru...| Negative|   274|
|1/25 Part 1 - Why...| Negative|   300|
|For those self in...| Negative|   166|
|Report any and al...| Negative|   214|
|Met with members ...| Negative|   303|
|Every shop, chemi...| Negative|   270|
+--------------------+---------+------+
only showing top 10 rows



In [10]:
data = data.na.drop()
data.show()

+--------------------+--------------------+------+
|       OriginalTweet|           Sentiment|length|
+--------------------+--------------------+------+
|TRENDING: New Yor...|  Extremely Negative|   228|
|When I couldn't f...|            Positive|   193|
|Find out how you ...|  Extremely Positive|    73|
|#toiletpaper #dun...|             Neutral|   252|
|Do you remember t...|             Neutral|   205|
|Voting in the age...|            Positive|    90|
|HI TWITTER! I am ...|  Extremely Negative|   280|
|Anyone been in a ...|  Extremely Positive|   238|
|While we were bus...|            Positive|   197|
|When youre stock...|             Neutral|   187|
|#CoronaVirus #COV...|            Negative|   284|
|"For those of you...|  Extremely Positive|   279|
|Control over stoc...|  Extremely Positive|   237|
|When your usual g...|            Negative|   201|
|While you are sto...|            Negative|   250|
|"#Coronavirus is ...|"" says NELP Gove...|    72|
|Two important pho...|         

In [11]:
data = data.filter("Sentiment == 'Neutral' OR Sentiment == 'Negative' OR Sentiment == 'Positive' OR Sentiment == 'Extremely Negative' OR Sentiment == 'Extremely Positive'")
data.show()

+--------------------+------------------+------+
|       OriginalTweet|         Sentiment|length|
+--------------------+------------------+------+
|TRENDING: New Yor...|Extremely Negative|   228|
|When I couldn't f...|          Positive|   193|
|Find out how you ...|Extremely Positive|    73|
|#toiletpaper #dun...|           Neutral|   252|
|Do you remember t...|           Neutral|   205|
|Voting in the age...|          Positive|    90|
|HI TWITTER! I am ...|Extremely Negative|   280|
|Anyone been in a ...|Extremely Positive|   238|
|While we were bus...|          Positive|   197|
|When youre stock...|           Neutral|   187|
|#CoronaVirus #COV...|          Negative|   284|
|"For those of you...|Extremely Positive|   279|
|Control over stoc...|Extremely Positive|   237|
|When your usual g...|          Negative|   201|
|While you are sto...|          Negative|   250|
|Two important pho...|          Negative|   271|
|IMHO Russia's rug...|Extremely Negative|   280|
|So not due to #Br..

In [12]:
data = data.withColumn('Sentiment', regexp_replace('Sentiment', 'Extremely Negative', 'Negative'))
data = data.withColumn('Sentiment', regexp_replace('Sentiment', 'Extremely Positive', 'Positive'))

In [13]:
data.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|TRENDING: New Yor...| Negative|   228|
|When I couldn't f...| Positive|   193|
|Find out how you ...| Positive|    73|
|#toiletpaper #dun...|  Neutral|   252|
|Do you remember t...|  Neutral|   205|
|Voting in the age...| Positive|    90|
|HI TWITTER! I am ...| Negative|   280|
|Anyone been in a ...| Positive|   238|
|While we were bus...| Positive|   197|
|When youre stock...|  Neutral|   187|
|#CoronaVirus #COV...| Negative|   284|
|"For those of you...| Positive|   279|
|Control over stoc...| Positive|   237|
|When your usual g...| Negative|   201|
|While you are sto...| Negative|   250|
|Two important pho...| Negative|   271|
|IMHO Russia's rug...| Negative|   280|
|So not due to #Br...|  Neutral|   148|
|Fellow Uni instru...| Negative|   274|
|1/25 Part 1 - Why...| Negative|   300|
+--------------------+---------+------+
only showing top 20 rows



# Features Transformation

In [14]:
tokenizer=Tokenizer(inputCol="OriginalTweet", outputCol="token_text")
stopremove=StopWordsRemover(inputCol="token_text", outputCol="stop_tokens")
count_vec=CountVectorizer(inputCol="stop_tokens", outputCol="c_vec")
#hashingTF = HashingTF(inputCol="stop_tokens", outputCol="c_vec", numFeatures=5000)
idf=IDF(inputCol="c_vec", outputCol="tf_idf")
# we also need to convert our labels in numbers
ham_samp_to_num = StringIndexer(inputCol="Sentiment", outputCol='label')
clean_up = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')

# Model

In [216]:
dtc=DecisionTreeClassifier(maxDepth=30)
rf=RandomForestClassifier(numTrees=200)
nb=NaiveBayes()

# Pipeline

In [164]:
data_prep_pipeline= Pipeline(stages=[ham_samp_to_num, tokenizer, stopremove,count_vec, idf,clean_up, nb])

In [165]:
data.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|TRENDING: New Yor...| Negative|   228|
|When I couldn't f...| Positive|   193|
|Find out how you ...| Positive|    73|
|#toiletpaper #dun...|  Neutral|   252|
|Do you remember t...|  Neutral|   205|
|Voting in the age...| Positive|    90|
|HI TWITTER! I am ...| Negative|   280|
|Anyone been in a ...| Positive|   238|
|While we were bus...| Positive|   197|
|When youre stock...|  Neutral|   187|
|#CoronaVirus #COV...| Negative|   284|
|"For those of you...| Positive|   279|
|Control over stoc...| Positive|   237|
|When your usual g...| Negative|   201|
|While you are sto...| Negative|   250|
|Two important pho...| Negative|   271|
|IMHO Russia's rug...| Negative|   280|
|So not due to #Br...|  Neutral|   148|
|Fellow Uni instru...| Negative|   274|
|1/25 Part 1 - Why...| Negative|   300|
+--------------------+---------+------+
only showing top 20 rows



In [166]:
(training, testing)= data.randomSplit([0.8,0.2])

In [167]:
training.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
| Food poverty doe...| Negative|   225|
| There is enough ...|  Neutral|    97|
| This is the best...| Positive|   181|
| and major tech c...| Positive|    49|
| and vegetables&g...| Positive|    30|
| as bad as that is."| Negative|    20|
| we shouldn't pan...| Negative|    21|
|"""Current estima...|  Neutral|   138|
|"#Airbnb some hos...| Negative|   287|
|"#BREAKING: Ontar...| Negative|   250|
|"#Covid_19 #Hamst...| Positive|   207|
|"2 Weeks #Lockdow...|  Neutral|   307|
|"?? Starbucks may...| Positive|   127|
|"@AskTarget ""cor...| Positive|   244|
|"@Ryanair is at i...| Positive|   259|
|"@iamshafaatali I...| Negative|   259|
|"@phineashead @Bo...| Negative|   259|
|"@quietplacemovie...| Positive|   245|
|"A ""consumer rem...| Positive|   259|
|"Apparently Chest...| Positive|   174|
+--------------------+---------+------+
only showing top 20 rows



In [168]:
testing.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|  We ve got sever...| Negative|   235|
| Amazon Needs to ...| Negative|   203|
|!!!!!!! French Pr...| Negative|   133|
|"@colemank83 offe...| Positive|   221|
|"@yogagirlmab Ano...| Negative|   261|
|"Crowe's Gala in ...| Positive|   199|
|"For those of you...| Positive|   279|
|"I usually stock ...| Negative|   306|
|"NO #toiletpaper ...| Negative|   264|
|#BBCqt @MattHanco...|  Neutral|    70|
|#Coronavirus bori...| Negative|   225|
|#Coronavirus has ...| Positive|   277|
|#Coronavirus, or ...| Positive|   137|
|#Coronavirus: L.A...|  Neutral|   112|
|#CoronavirusPande...| Positive|   223|
|#CoronavirusPande...| Positive|   218|
|#CoronavirusPande...| Positive|   302|
|#Covid_19 All the...|  Neutral|   209|
|#Covid_19 Third, ...| Negative|   263|
|#Covid_19 is in t...| Positive|    65|
+--------------------+---------+------+
only showing top 20 rows



In [169]:
%%time
model = data_prep_pipeline.fit(training)

CPU times: user 34.6 ms, sys: 23.1 ms, total: 57.8 ms
Wall time: 2.92 s


In [170]:
model

PipelineModel_d9ce09ee9091

### Naive Bayes

In [171]:
dx = spark.createDataFrame(testing.take(2))

In [172]:
dx.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|  We ve got sever...| Negative|   235|
| Amazon Needs to ...| Negative|   203|
+--------------------+---------+------+



In [173]:
test_results=model.transform(testing)

In [174]:
test_results.show()

+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|       OriginalTweet|Sentiment|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|       rawPrediction|         probability|prediction|
+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  We ve got sever...| Negative|   235|  0.0|[, , we, ve, got,...|[, , ve, got, sev...|(11867,[2,3,24,37...|(11867,[2,3,24,37...|(11868,[2,3,24,37...|[-1105.1072412442...|[4.50602241541471...|       1.0|
| Amazon Needs to ...| Negative|   203|  0.0|[, amazon, needs,...|[, amazon, needs,...|(11867,[2,7,9,10,...|(11867,[2,7,9,10,...|(11868,[2,7,9,10,...|[-1182.4923253025...|[9.1749154709

In [175]:
acc_eval=MulticlassClassificationEvaluator()
acc=acc_eval.evaluate(test_results)

In [176]:
print ("Accuracy of the model is::", acc)

Accuracy of the model is:: 0.5307387964171865


### Decision Tree Classifier

In [177]:
data_prep_pipeline= Pipeline(stages=[ham_samp_to_num, tokenizer, stopremove,count_vec, idf,clean_up, dtc])

In [178]:
model = data_prep_pipeline.fit(training)
model

PipelineModel_bb5e3c42ed37

In [179]:
dx = spark.createDataFrame(testing.take(2))
dx.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|  We ve got sever...| Negative|   235|
| Amazon Needs to ...| Negative|   203|
+--------------------+---------+------+



In [180]:
test_results=model.transform(testing)
test_results.show()

+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+----------+
|       OriginalTweet|Sentiment|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|    rawPrediction|         probability|prediction|
+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+----------+
|  We ve got sever...| Negative|   235|  0.0|[, , we, ve, got,...|[, , ve, got, sev...|(11867,[2,3,24,37...|(11867,[2,3,24,37...|(11868,[2,3,24,37...|   [36.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
| Amazon Needs to ...| Negative|   203|  0.0|[, amazon, needs,...|[, amazon, needs,...|(11867,[2,7,9,10,...|(11867,[2,7,9,10,...|(11868,[2,7,9,10,...|[125.0,81.0,58.0]|[0.47348484848484...|       

In [181]:
acc_eval=MulticlassClassificationEvaluator()
acc=acc_eval.evaluate(test_results)

In [182]:
print ("Accuracy of the model is::", acc)

Accuracy of the model is:: 0.5258182243389363


### Random Forest

In [217]:
data_prep_pipeline= Pipeline(stages=[ham_samp_to_num, tokenizer, stopremove,count_vec, idf,clean_up, rf])

In [218]:
model = data_prep_pipeline.fit(training)
model

PipelineModel_10feb5616eaa

In [219]:
dx = spark.createDataFrame(testing.take(2))
dx.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|  We ve got sever...| Negative|   235|
| Amazon Needs to ...| Negative|   203|
+--------------------+---------+------+



In [220]:
test_results=model.transform(testing)

In [221]:
test_results.show()

+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|       OriginalTweet|Sentiment|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|       rawPrediction|         probability|prediction|
+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  We ve got sever...| Negative|   235|  0.0|[, , we, ve, got,...|[, , ve, got, sev...|(11867,[2,3,24,37...|(11867,[2,3,24,37...|(11868,[2,3,24,37...|[92.0167193125639...|[0.46008359656281...|       0.0|
| Amazon Needs to ...| Negative|   203|  0.0|[, amazon, needs,...|[, amazon, needs,...|(11867,[2,7,9,10,...|(11867,[2,7,9,10,...|(11868,[2,7,9,10,...|[87.0097154493838...|[0.4350485772

In [222]:
acc_eval=MulticlassClassificationEvaluator()
acc=acc_eval.evaluate(test_results)

In [223]:
print ("Accuracy of the model is::", acc)

Accuracy of the model is:: 0.3161533537953037
